In [3]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 740.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=71fe01c859e41ddee6b4cc4a3605537e52adc7ea9ebb170c2fb5dd2a21dde491
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0

In [9]:
from googletrans import Translator

# Create a translator object
translator = Translator()

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

import transformers
transformers.__version__

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline('text-classification', model=finbert, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'Positive', 'score': 0.9999747276306152}]


In [11]:
results = nlp("I am happy")
print(results)

[{'label': 'Positive', 'score': 0.9999747276306152}]


In [12]:
import gspread
from google.auth import default
from google.colab import auth

# Authenticate and authorize access to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet and select the worksheet
spreadsheet_id = '1fZwXtdgPClD43DjP4Rn2Th_bw76RU8HB6tV67oQcLxI'
worksheet_name = 'nikkei_headlines.csv'  # Replace with the actual sheet name
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name)

# Iterate through the results and update the spreadsheet
row_index = 1  # Start from row 1 (the first row)

In [13]:
# Read the values in column C
column_c_values = worksheet.col_values(3)  # 3 represents column C

In [21]:
len(column_c_values)

29323

In [ ]:
from tqdm import tqdm

for row_index, headline in enumerate(tqdm(column_c_values)):
  # if row_index == 0:
  #   print(row_index)
  if row_index <= 2298:
    continue
  #   break

  text_to_translate = column_c_values[row_index+1] # row_index is 0-idx, but access to gsheet is 1-idx

  translated = translator.translate(text_to_translate, src='ja', dest='en')
  # print(translated.text)

  result = nlp(translated.text)

  label = result[0]['label']
  score = result[0]['score']

  worksheet.update_cell(row_index+1, 4, label)  # Column D is index 4 (1-indexed)
  worksheet.update_cell(row_index+1, 5, score)  # Column E is index 5 (1-indexed)
  worksheet.update_cell(row_index+1, 6, translated.text)  # Column F is index 6 (1-indexed)